<a href="https://colab.research.google.com/github/Bast-94/ML-BIO/blob/drafts-molecule/1_molecules_representations_etudiants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install rdkit
!pip install rdkit-pypi

## Ex 1. Read molecules from SMILES

SMILES (Simplified Molecular-Input Line-Entry System) is a popular method for specifying molecules with text string. The RDkit library provides utilities to read molecules from SMILES.


Paracetamol (Acetaminophen) is a commonly used medicine that can help treat pain and reduce a high temperature. It's given by the chemical formula $ C_8H_9N O_2$ and its SMILES is "CC(=O)NC1=CC=C(O)C=C1".


1. Create the paracetamol molecule from its SMILES using the rdkit.Chem.MolFromSmiles class.


2. Visualize its structure (it's sufficient to print the paracetamol molecule or to use rdkit.Chem.Draw.MolToImage())


3. Print out the number of atoms. Did you get 11 ? It should be 20 atoms ? Why ?



4. Print out the list of atoms.


5. Find the number of bonds.



*1. Create the paracetamol molecule from its SMILES using the rdkit.Chem.MolFromSmiles class.*

In [ ]:
from rdkit.Chem import MolFromSmiles
from rdkit.Chem.Draw import MolToImage
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
from rdkit.Chem import Descriptors
import pandas as pd
from rdkit.Chem import SDMolSupplier
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
import numpy as np
from rdkit.DataStructs import DiceSimilarity
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

In [ ]:
paracetamol = MolFromSmiles("CC(=O)NC1=CC=C(O)C=C1")

*2. Visualize its structure*

In [ ]:
paracetamol_img = MolToImage(paracetamol)
fig, ax = plt.subplots()
ax.set_xticks([])
ax.set_yticks([])
ax.imshow(paracetamol_img)
plt.show()

*3. Print out the number of atoms. Did you get 11 ? It should be 20 atoms ? Why ?*

Vérifions avec la méthode `GetNumAtoms`

In [ ]:
number_of_atoms = paracetamol.GetNumAtoms()
print("Number of atoms: ", number_of_atoms)

Nous obtenous effectivement 11, nous pouvons supposer que `GetNumAtoms` ommet volontairement les atomes d'hydrogène. Regardons la liste des atomes du paracétamol avec la méthode `GetAtoms`.

*4. Print out the list of atoms.*

In [ ]:
for atom in paracetamol.GetAtoms():
    print(atom.GetSymbol(), end=" ")

On remarque effectivement que les atomes d'hydrogène ne sont pas présents, ce qui justifie le fait que seul 11 atomes soient comptés.

*5. Find the number of bonds.*

In [ ]:
bonds_number = paracetamol.GetNumBonds()
print(f"Il y a {bonds_number} liaisons")

## Ex 2 Read data from chemical file format

Apart SMILES, .SDF/ .Mol are also the common formats to save molecules. RDKit provides also functionalities to read these files.


Morphine is an opioid agonist used for the relief of moderate to severe acute and chronic pain. The chemical information of this drug can be found at
"https://go.drugbank.com/structures/small_molecule_drugs/DB00295.sdf".


**1. Download this file and save it in a folder.**

In [ ]:
! wget https://go.drugbank.com/structures/small_molecule_drugs/DB00295.sdf -o log.txt -O DB00295.sdf
! test -f DB00295.sdf && echo "File downloaded" || echo "File not downloaded"
! rm log.txt

**2. Read the morphine molecule from the file that you've downloaded using "rdkit.Chem.SDMolSupplier"**

In [ ]:
suppl = SDMolSupplier("DB00295.sdf")
fig = plt.figure(figsize=(len(suppl) * 5, 5))
fig.set_facecolor("white")
for i, mol in enumerate(suppl):
    ax = fig.add_subplot(1, len(suppl), i + 1)
    molimg = MolToImage(mol)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.imshow(molimg)
    ax.set_title(mol.GetProp("GENERIC_NAME"))
    plt.show()

**3. With the help of rdkit.Chem.AllChem.GetMorganFingerprintAsBitVect, compute its fingerprint as a bit vector.**

In [ ]:
morphine = suppl[0]
bit_vector = np.array(GetMorganFingerprintAsBitVect(morphine, 2, nBits=1024))
assert np.all((bit_vector == 0) | (bit_vector == 1))
bit_vector.shape

## EX3. Fingerprints


A chemical structure can be characterized by a set of numerical values known as molecular fingerprints. They may be 2D Fragment-based fingerprints presented by bit arrays of 0s and 1s wherein each bit position indicates the presence or absence of structural fragments.


The RDKit has a variety of built-in functionality for generating molecular fingerprints and then using them to calculate molecular similarity. In this exercise, we will generate the fingerprints for parcetamol and aspirin that are two popular drugs used for pains.



**1. Compute the fingerprint of the paracetamol as bit vector by using the rdkit.AllChem.GetMorganFingerprintAsBitVectCompute Class. Convert it to a numpy vector. Known that Paracetamol's SMILES is ""CC(=O)OC1=CC=CC=C1C(O)=O". Choose the number of bits nBits= 1024. Find the elements that are nonzero of the vector.**


In [ ]:
def get_non_zero(molecule):
    bit_vector = np.array(GetMorganFingerprintAsBitVect(molecule, 2, nBits=1024))
    return np.nonzero(bit_vector)

In [ ]:
paracetamol = MolFromSmiles("CC(=O)NC1=CC=C(O)C=C1")
get_non_zero(paracetamol)

**2. Do the same thing for the aspirin. Aspirin's SMILES is "CC(=O)NC1=CC=C(O)C=C1".**

In [ ]:
aspirin = MolFromSmiles("CC(=O)OC1=CC=CC=C1C(=O)O")
get_non_zero(aspirin)

**3. Using rdkit.DataStructs.DiceSimilarity(), compute the DiceSimilarity between the fingerprints of the paracetamol and the aspirin.**

In [ ]:
paracetamol_bit_vector = GetMorganFingerprintAsBitVect(paracetamol, 2, nBits=1024)
aspirin_bit_vector = GetMorganFingerprintAsBitVect(aspirin, 2, nBits=1024)
DiceSimilarity(paracetamol_bit_vector, aspirin_bit_vector)

## EX 4: Descriptors

Molecules are also described by a set of physiochemical descriptors. They should be the log partition coefficient, the polar surface area ... that describe the molecule's structure.


RDKit provides utilities that allow us to compute descriptor of molecules.
Using the rdkit.ML.Descriptors.MoleculeDescriptors.MolecularDescriptorCalculator class, compute the descriptor for paracetamol molecule from SMILES string. Note that SMILES string for paracetamol is "CC(=O)NC1=CC=C(O)C=C1"



In [ ]:
paracetamol = MolFromSmiles("CC(=O)NC1=CC=C(O)C=C1")
desc_names = [desc_name for desc_name, _ in Descriptors.descList]
description = MolecularDescriptorCalculator(desc_names).CalcDescriptors(paracetamol)
df = pd.DataFrame({"Descriptor": desc_names, "Value": description})
df.head()

## Ex5: SMART

In many commonly used applications, we need to determine whether atoms in a molecule match a particular pattern. It can be used for filtering structures or identifying substructures that are associated with toxicological problem.

SMART us an extension of the SMILES language that can be used to create queries.

1. Find molecules in the following list named "smiles_list" that contain the "CCC" pattern (you can use the GetSubstructMatch class).

In [ ]:
smiles_list = [
    "CCCCC",
    "CCOCC",
    "CCNCC",
    "CC(=O)NC1=CC=C(O)C=C1",
    "CC(=O)NC1=CC=C(O[13CH3])C=C1",
    "CN(C(=O)C(Cl)Cl)C1=CC=C(O)C=C1",
]
fig = plt.figure(figsize=(len(smiles_list) * 5, 5))
fig.set_facecolor("white")

pattern = Chem.MolFromSmarts("CCC")
mol_list = [MolFromSmiles(smiles) for smiles in smiles_list]
for i, mol in enumerate(mol_list):
    ax = fig.add_subplot(1, len(smiles_list), i + 1)

    ax.set_xticks([])
    ax.set_yticks([])
    substructs = mol.GetSubstructMatches(pattern)
    molimg = MolToImage(mol)
    ax.imshow(molimg)
    ax.set_title(f"{Chem.MolToSmiles(mol)}")

    if mol.HasSubstructMatch(pattern):
        print(f"Match for {Chem.MolToSmiles(mol)}")
    else:
        print(f"No match for {Chem.MolToSmiles(mol)}")

plt.tight_layout()
plt.show()

Nous remarquons qu'une seul molécule correspond au pattern `CCC` c'est la molécule à la notation SMILE `CCCCC`.

**2. Highlight the pattern "CCC" in these molecules**

In [ ]:
matching_mol = mol_list[0]
matching_mol.GetSubstructMatches(pattern)
matching_mol

**3. Do the same thing ex 1 but for the pattern "C(=O)N".**

## Ex6: Machine learning with RDkit - Predict solubility

In this exercise, we will use machine learning to predict solubility of molecules. For this purpose, we will borrow a dataset from rdkit that is originated from the Huuskonen dataset. We will try to predict Aqueous Solubility for molecules that is known as "logS".


This exercise is divided into 3 parts: Preparing a dataset, training a model and preparing dataset for prediction and applying a predictive model.



## EX6.I Preparing dataset

Preparing dataset requires a few steps:


**1. Download file from "https://raw.githubusercontent.com/rdkit/rdkit/master/Docs/Book/data/solubility.train.sdf".**

In [ ]:
![ -e solubility.train.sdf ] || wget https://raw.githubusercontent.com/rdkit/rdkit/master/Docs/Book/data/solubility.train.sdf


**2. With the help of rdkit.Chem.SDMolSupplier, get list of molecules contained in this file. You should call this list as **molecule_list**. The option **removeHs=False** should be chosen.**

In [ ]:
molecule_list = SDMolSupplier(fileName="solubility.train.sdf", removeHs=False)

In [ ]:
print(f"Il y a {len(molecule_list)} molécules dans le fichier")


**3. Write a function named **calculate_descriptors(mol)** that allows us to calculate descriptors of a molecule. This function takes a Rdkit molecule as input and returns an array vector of descriptors.**

In [ ]:
def calculate_descriptor(molecule):
    desc_names = [desc_name for desc_name, _ in Descriptors.descList]
    description = MolecularDescriptorCalculator(desc_names).CalcDescriptors(molecule)
    return np.array(description)

**4. Apply the function **caluclate_descriptors** to the list of molecules **molecule_list** and store the result in a dataframe named **df**. Look at few rows of **df** to see whether the descriptors are calculated.**

In [ ]:
df_dict = {}
n_features = len(Descriptors.descList)
for mol in molecule_list:
    df_dict[mol.GetProp("NAME")] = calculate_descriptor(mol)

df = pd.DataFrame(df_dict).T

In [ ]:
df.head()

**5. For each molecule from this dataset, we can get the aqueous solubility (logS) via attribute **getProp('SOL')**. Create a list that contains the aqueous solubility of all molecules from the **molecule_list** list. You should call this list by **labels**. Remember to convert these values to float format.**

In [ ]:
labels = [float(mol.GetProp("SOL")) for mol in molecule_list]

In [ ]:
labels[:5]



**6. Plot the histogram of the "labels" list to see the distribution of the solubility of molecules. Do you have some comments about the aqueous solubility of molecules ?**



In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.hist(labels, bins=50)
ax.set_xlabel("Solubility")
ax.set_ylabel("Nombre de molecules")
ax.set_title("Distribution de la solubilité")
plt.show()

On peut remarquer que la distribution des valeurs de la log-solubilité est centrée autour de -2. On peut donc supposer que la majorité des molécules sont peu soluble dans l'eau.

## Ex6.II Feature Engineering and Training a model of regression.

Now, we have the **df** dataframe that contains the descriptors for molecules and the **labels** list that contains the solvant property of molecules.

1. Check the dataframe **df** to see whether it contains NaN values. How many row contained NaN values are there in the dataframe **df** ? Remove these rows from  **df** and **labels**.

In [ ]:
df["labels"] = labels

In [ ]:
np.isnan(df).any().any()

Le dataframe contient des valeurs NaN. Nous allons donc les supprimer.

In [ ]:
df.dropna(inplace=True)
assert not np.isnan(df).any().any()

2. Apply the MinMaxScaler to the dataframe **df** to normalize the data.

In [ ]:
scaler = StandardScaler()
scaler.fit(df.values[:, :-1])
scaled_data = scaler.transform(df.values[:, :-1])
assert scaled_data.shape[1] == (n_features)

3. Construct and train a regression model.

In [ ]:
X = scaled_data
Y = df["labels"].values
assert X.shape[0] == Y.shape[0]



4. Calculate the square root error for the dataset

In [ ]:
model = Lasso(alpha=0.01)
model.fit(X, Y)
single_Y_pred = model.predict(X)
square_root_error = np.sqrt(np.mean((single_Y_pred - Y) ** 2))
square_root_error

## Ex6.III Prepare data for test  and test model


1. Download file from "https://github.com/rdkit/rdkit/blob/master/Docs/Book/data/solubility.test.sdf".

In [ ]:
# 1. Download the file
![ -e solubility.test.sdf ] || wget https://raw.githubusercontent.com/rdkit/rdkit/master/Docs/Book/data/solubility.test.sdf


2. Read molecules from this file and store them in a list named **list_molecule_test**.

In [ ]:
list_molecule_test = SDMolSupplier(fileName="solubility.test.sdf")


3. Take a molecule from the **list_molecule_test** and then calculate its descriptions thank to the **calculate_descriptors(mol)** function that you've coded above.

In [ ]:
import random

chosen_index = random.randint(0, len(list_molecule_test))
molecule = list_molecule_test[chosen_index]
descriptors = calculate_descriptor(molecule)

In [ ]:
descriptors.reshape(1, -1).shape

4. Apply **minmaxscaler** to these descriptors. Note that **minmaxscaler**  is one that you've created at Ex6.II.

In [ ]:
descriptors = scaler.transform(descriptors.reshape(1, -1))

5. Use the model that you've trained to predict the aqueous solubility (logS) of the molecule. Compare to the real logS value of the molecule.

In [ ]:
single_Y_test = model.predict(descriptors)
single_Y_pred = molecule.GetProp("SOL")
print(f"Predicted: {single_Y_test[0]:.2f} | Real: {single_Y_pred}")


6. (Optional) Calculate the mean square root error of the model for this dataset.

Note: If there exists problem of reading the file solubility.test.sdf as
"RDKit ERROR: [09:28:36] ERROR: moving to the beginning of the next molecule
RDKit ERROR: [09:32:48] ERROR: Counts line too short: '' on line4"
so, go to the url, click "raw" and save this file by hand. The error will be gone away

In [ ]:
X_test = np.array([calculate_descriptor(mol) for mol in list_molecule_test])
X_test = scaler.transform(X_test)
y_test = np.array([float(mol.GetProp("SOL")) for mol in list_molecule_test])
assert X_test.shape[0] == y_test.shape[0]
y_pred_test = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred_test)
rmse = np.sqrt(mse)
print(f"RMSE : {rmse:.2f}")